In [1]:
#======================================================================================================

import ipywidgets as w
from IPython.display import display, clear_output, Image, IFrame
import numpy as np
import re
import pandas as pd
import pyperclip

_canvas_rows = w.Text(layout = w.Layout(width='70px'), placeholder = 'e.g., 3')
_canvas_cols = w.Text(layout = w.Layout(width='70px'), placeholder = 'e.g., 4')

_create_canvas = w.Button(icon = 'check', layout = w.Layout(width = '35px'))


_canvas_button_colors = ['#008080FF',
                         '#DEB887FF',
                         '#F08080FF',
                         '#87CEEBFF',
                         '#40E0D0FF',
                         '#EEE8AAFF']*10

_graph_types = 'Table, Scatter, Bar'.split(', ')

_element_type = w.Dropdown(options = _graph_types, layout=w.Layout(width = '80px'))

_add_graph = w.Button(description = 'add', layout=w.Layout(width = '60px'), disabled=True)

_restart_canvas = w.Button(description = 'Restart', icon = 'refresh', layout=w.Layout(width = '100px'))

_graphs_out = w.Output(layout=w.Layout(height='895px',
                              width='99.5%',
                              border='12px inset teal',
                              align_items='center'))

_graphs_top_out = w.Output(layout=w.Layout(height = '60px', width = '95.5%',
                                          align_items='center', justify_content='center'))

_canvas_out = w.Output(layout = w.Layout(height='805px', align_items='center', width = '99.5%',
                                         justify_content='center'))

#======================================================================================================

with _graphs_top_out:
    clear_output(True)
    display(w.HBox([w.HTML('<b><h2><font color = cadetblue>Graph type: '), _element_type, _add_graph,
                   w.HTML(value="<h4><b><font color='black'>｜"), _restart_canvas], 
                   layout=w.Layout(align_items='center', justify_content='center')))
    
with _graphs_out:
    display(w.VBox([w.HTML('<b><font color = grey><h4>Create a'),
                    w.HBox([_canvas_rows, w.HTML('<b><font color = grey><h4>x'), _canvas_cols],
                          layout = w.Layout(align_items='center')),
                    w.HBox([w.HTML('<b><font color = grey><h4>canvas'), _create_canvas],
                          layout=w.Layout(align_items='center'))],
                   layout=w.Layout(align_items='center')))

#========================================================================================================

def _check_isdigit(a):
    
    global _rows, _columns
    
    if not all([_canvas_rows.value.isdigit(), _canvas_cols.value.isdigit()]) and \
       (_canvas_rows.value != '' and _canvas_cols.value != ''):
        
        _canvas_rows.value = ''
        _canvas_cols.value = ''
        
        _create_canvas.on_click(_create_canvas_func, remove=True)
        
    else:
        
        if _canvas_rows.value == '' and _canvas_cols.value == '':

            _rows = 3
            _columns = 4

        else:

            _rows = int(_canvas_rows.value)
            _columns = int(_canvas_cols.value)
            
        _create_canvas.on_click(_create_canvas_func)
        
_create_canvas.on_click(_check_isdigit)

#===========================================

def _restart_canvas_func(a):
    
    _canvas_rows.value = ''
    _canvas_cols.value = ''
    
    with _graphs_out:
        clear_output(True)
        display(w.VBox([w.HTML('<b><font color = cadetblue><h2>Create a'),
                    w.HBox([_canvas_rows, w.HTML('<b><font color = grey><h2>x'), _canvas_cols],
                          layout = w.Layout(align_items='center')),
                    w.HBox([w.HTML('<b><font color = cadetblue><h2>canvas'), _create_canvas],
                          layout=w.Layout(align_items='center'))],
                   layout=w.Layout(align_items='center')))
    
_restart_canvas.on_click(_restart_canvas_func)

In [2]:
#========================================================================================================

global _create_canvas_func

def _create_canvas_func(a):
    
    global _rows, _columns
    
    #---------------------------
    
    # Each time a legitimate area is submitted as a graph, the counter will advance, which will be used for
    # naming the graphs (i.e. Figure 1, Figure 2 etc) and other maintenance reasons
    
    global _counter 
    _counter = 0

    #---------------------------
    
    # dict that will capture modulo for each button, see below
    
    global _modulo_dict
    _modulo_dict = {}
    
    #---------------------------
    
    global _specs_graphs # list of graph types, ie., [Table, Bar, Scatter] for the final specs
    _specs_graphs = []
    
    global _specs_elements # individual elements of the specs argument for plotly
    _specs_elements = ['' for i in range(_rows*_columns)]

    global _final_specs # final specs argument for plotly based on specs_elements
    _final_specs = ['' for i in  range(_rows)]
    
    #---------------------------
    
    global _row_col_start # list of row/col tuples representing row/col start for each graph
    _row_col_start = []
    
    #----------------------------------------------------------------------------------------------

    # Make and display canvas
    
    width = str(99.5/int(_columns))
    height = str((780 - 2*(_rows))/int(_rows))

    canvas_children = [] # children for the whole canvas

    start = 1
    end = _columns +1

    for j in range(_rows):
        
        globals()['_canvas_children_' + str(j+1)] = [] # children for each _rows of buttons
        
        for i in range(start, end):

            globals()["_Sel_" + str(i)] = w.Button(layout = w.Layout(
                                            width = width + '%', height = height + 'px', margin="2px"), 
                                            style={'button_color':'gainsboro'})

            globals()['_canvas_children_' + str(j+1)].append(globals()["_Sel_" + str(i)])

        start+= _columns
        end+= _columns

        canvas_children.append(globals()['_canvas_children_' + str(j+1)])

    #---------------------------
    
    # iterate through all the buttons and determine what happens when it is pressed
    # i.e., which other buttons get highlighted and which ones don't
    # this will be determined using _modulo_dict
    # for example, pressing button 4 will highlight buttons 1, 2, 3 and 5, 6, 7 (eg. 3x4 grid)
    
    for i in np.arange(_columns*_rows) + 1:
        
        #---------------------------
        
        # create _modulo_dict
        
        modulo = i%_columns
        
        if modulo == 0:
            modulo = _columns
        
        _modulo_dict[i] = modulo # modulo for each button in the canvas
        
        #---------------------------
        
        # for each button, determine which buttons get highlighted darkgrey
        
        ls = []

        for j in range(1, modulo+1):
            ls.append([i for i in _modulo_dict.keys() if _modulo_dict[i] == j])

        ls = sorted([item for k in ls for item in k])

        # all the other buttons that should not be highlighted
        non_ls = sorted([i for i in list(np.arange(_rows*_columns)+1) if i not in ls])

        # for each button, create function for highlighting relevant buttons

        string = 'global _Sel_' + str(i) + '_func' + '\n'  +   \
                 'def _Sel_' + str(i) + '_func(a):' + '\n' + \
                 '\tglobal _to_commit' + '\n' + \
                 '\t_to_commit = ' + ''.join(str(ls)) + '\n' + \
                 '\tfor k in ' + ''.join(str(ls)) + ':' + '\n' + \
                 '\t\tglobals()["_Sel_" + str(k)].style.button_color = "' + 'darkgrey' + '"\n' + \
                 '\tfor k in ' + ''.join(str(non_ls)) + ':' + '\n' + \
                 '\t\tglobals()["_Sel_" + str(k)].style.button_color = "gainsboro"' + '\n' + \
                 '\t_add_graph.disabled = False' + '\n' + \
                 '_Sel_' + str(i) + '.on_click(_Sel_' + str(i) + '_func)'    

        exec(string)
    
    #---------------------------
    
    # Show canvas
    
    with _graphs_top_out:
        clear_output(True)
        display(w.HBox([w.HTML('<b><h2><font color = cadetblue>Graph type: '), _element_type, _add_graph,
                   w.HTML(value="<h4><b><font color='black'>｜"), _restart_canvas], 
                   layout=w.Layout(align_items='center', justify_content='center')))
    
    with _canvas_out:
        clear_output(True)
        display(w.VBox([w.HBox(i, layout=w.Layout(grid_gap = '0.5px')) for i in canvas_children], 
                       layout=w.Layout(grid_gap = '0%')))

    with _graphs_out:
        clear_output(True)
        display(w.HBox([w.Output(layout=w.Layout(width = '1.6px')),
            w.VBox([_graphs_top_out, _canvas_out], layout=({'width':'99.5%'}))]))
        
_create_canvas.on_click(_create_canvas_func)

In [3]:
# ========================================================================================================

# Begin capturing areas for figures

def _add_graph_func(a):
    
    global _counter, _seq
        
    #---------------------------
    
    # sequence of buttons that should be highlighted
    
    if _counter == 0: # when the very first figure is being made 
        
        _seq = _to_commit # to_commit is determined in create_canvas_func
        
    else: 
        
        # it is possible that some of the buttons that are eligible for coloring and those that are
        # already colored in the previous step overlap. Do not recolor those that are already colored
        
        _seq = [i for i in _to_commit if i not in _colored_buttons] # _colored_buttons is determined in _non_colored_buttons_func
    
    #---------------------------
    
    # when add_graph is pressed, highlight the selected buttons and change their descriptions and assign
    # name for the top left corner button of the selected area, i.e. Fugure 1, Figure 2 etc
    
    for i in _seq:

        string = '_Sel_' + str(i) + ".description = ''"
        exec(string)
        string = '_Sel_' + str(i) + ".style.button_color = '" + _canvas_button_colors[_counter] + "'"
        exec(string)
        string = '_Sel_' + str(i) + ".layout.border = '4px solid " + _canvas_button_colors[_counter] + "'"
        exec(string)
        string = '_Sel_' + str(i) + ".style.font_size = '20px'"
        exec(string)
        string = '_Sel_' + str(min(_seq)) + ".description = '" + str(_counter+1) + "'"
        exec(string)
        string = '_Sel_' + str(min(_seq)) + ".style.font_size = 'auto'"
        exec(string)
        string = '_Sel_' + str(min(_seq)) + ".style.font_weight = 'bold'"
        exec(string)
    
    #---------------------------
    
    # advance the counter not each time add_graph button is pressed,
    # but when an actual area is selected and then add_graph is pressed
    # otherwise, figure names can be something like Figure 1, Figure 4, Figure 6 etc
    
    if len(_seq)>=1: 
        
        globals()['_Sel_' + str(min(_to_commit))].description = str(_counter+1) + ': ' + _element_type.value
        
        # add graph type to the list
        _specs_graphs.append(_element_type.value)
        _element_type.value = 'Table' # reset element type value each time a graph is added
        
        _counter+= 1   
    
    #---------------------------
    
    # after selecting the area for a figure and adding a Figure, remove functions associated with ALL 
    # the buttons (functions for non-colored graphs will be redefined below)
    
    for i in list(np.arange(_rows*_columns)+1):
        string = '_Sel_' + str(i) + '.on_click(_Sel_' + str(i) + '_func, remove=True)'
        exec(string)

_add_graph.on_click(_add_graph_func)

In [4]:
# ========================================================================================================

def _non_colored_buttons_func(a):
    
    global _colored_buttons
    
    #---------------------------
    
    # buttons that are currently colored
    _colored_buttons = [i for i in list(np.arange(_rows*_columns)+1) if globals()[
                         '_Sel_' + str(i)].style.button_color in _canvas_button_colors]
    
    # buttons that are not colored yet
    for i in [k for k in list(np.arange(_rows*_columns)+1) if k not in _colored_buttons]: 
                
        ls = [ # list of buttons that are eligible to be pressed, after the previous graph is made
                j for j in [
                    k for k in [
                        l for l in _modulo_dict.keys()
                        if _modulo_dict[l] <= _modulo_dict[i]
                    ] if k <= i
                ] if j not in _colored_buttons
              ]
        
        #---------------------------
        
        start = 1
        end = _columns + 1
        ls_count = []   # list of elements that fall on each row (e.g, graph covering 4, 5, 7, 8 in a 3x3 graph
                        # will result in [0, 2, 2])

        for j in range(_rows): # for each row count the number of elements that fall on it and append to ls_count

            ls_count.append(len([k for k in ls if k in range(start, end)]))

            start+= _columns
            end+= _columns
                                    
        if len(set([i for i in ls_count if i!=0]))==1: # if the selected area is symmetric
            
            non_ls = [i for i in sorted([i for i in list(np.arange(_rows*_columns)+1) if i not in ls]) if i not in _colored_buttons]

            string = 'global _Sel_' + str(i) + '_func' + '\n' + \
                     'def _Sel_' + str(i) + '_func(a):\n' + \
                     '\tglobal _to_commit' + '\n' + \
                     '\t_to_commit = ' + ''.join(str(ls)) + '\n' + \
                     '\tfor k in ' + ''.join(str(ls)) + ':' + '\n' + \
                     '\t\tglobals()["_Sel_" + str(k)].style.button_color = "' + 'darkgrey' + '"\n' + \
                     '\tfor k in ' + ''.join(str(non_ls)) + ':' + '\n' + \
                     '\t\tglobals()["_Sel_" + str(k)].style.button_color = "gainsboro"' + '\n' + \
                     '_Sel_' + str(i) + '.on_click(_Sel_' + str(i) + '_func)'

            exec(string)
             
        else:
            pass
    
    #---------------------------
    
    # disable add_graph button once all buttons are colored

    if len(_colored_buttons) == _rows*_columns:
        
        _add_graph.disabled=True
            
_add_graph.on_click(_non_colored_buttons_func)


In [5]:
# ========================================================================================================
        
def _create_graph_code_func(a):
    
    global _specs_elements
        
    #---------------------------
    
    if len(_seq)>=1: # only execute the below code when at least 1 button is selected
    
        # copy of dict_initial
        dict_column_span = _modulo_dict.copy()

        for i in np.arange(_rows)+1:
            dict_column_span[(i*_columns)] = _columns
        
        #---------------------------
        
        start = 1
        end = _columns + 1
        
        ls_count = []   # list of elements that fall on each row (e.g, graph covering 4, 5, 7, 8 in a 3x3 graph
                        # will result in [0, 2, 2])

        for j in range(_rows): # for each row count the number of elements that fall on it and append to ls_count

            ls_count.append(len([k for k in _to_commit if k in range(start, end)]))

            start+= _columns
            end+= _columns

        row_count = len([i for i in ls_count if i>0]) # how many rows does a figure cover
        col_count = set([i for i in ls_count if i>0]) # how many columns does a figure cover
        
        #---------------------------
        
        # make individual specs elements for plotly
        
        if len(_to_commit) == 1:

            _specs_elements[min(_to_commit)-1] = {}

        else:  

            _specs_elements[min(_to_commit)-1] = {'rowspan': row_count, 
                                                  'colspan': int(re.sub('[{}]',  '', str(col_count)))}

        _specs_elements = [None if x == '' else x for x in _specs_elements]
        
        #---------------------------
        
        # create final specs for plotly
        
        for num, i in enumerate([list(range(_rows*_columns)[i:i+_columns]) for i in range(
            0, _rows*_columns, _columns)]):
            _final_specs[num] = _specs_elements[min(i):max(i)+1]

        
        #---------------------------
        
        # determine on which row and column the graph starts
        
        row_start = []
        for num, i in enumerate([i > 0 for i in ls_count]):
                if i == True:
                    row_start.append(num)

        row_start = [i for i in row_start][0] + 1
        col_start = [val-1 for key,val in dict_column_span.items() if key == min(_to_commit)][0] + 1

        _row_col_start.append((row_start, col_start))

_add_graph.on_click(_create_graph_code_func)


In [6]:
# ========================================================================================================

def _add_graph_type_to_final_specs(a):
        
    counter = 0
    for num, i in enumerate(_final_specs):
        for num2, i in enumerate(i):
            if isinstance(i, dict):
                _final_specs[num][num2]['type'] = _specs_graphs[counter]
                counter += 1
                
_add_graph.on_click(_add_graph_type_to_final_specs)


In [7]:
# ========================================================================================================

def _copy_final_code(a):

    trace_code = []

    for num, i in enumerate(_specs_graphs):

        trace_code.append('# FIGURE ' + str(num+1) + '\nfig.add_trace(go.' + i + '(\n' + ' '*18 + 'x = ___, # column name\n' + \
                          ' ' *18 + 'y = ___), # row name\n' + ' '* 18 + 'row = ' + str(_row_col_start[num][0]) + \
              ', col = ' + str(_row_col_start[num][1]) + ')\n')


    pyperclip.copy(
        'import plotly.graph_objects as go\n' + 'from plotly.subplots import make_subplots\n\n\n' + \
        '# ---------------- SUBPLOT SETTINGS ---------------- #\n\n' + \
        'fig = make_subplots(rows = ' + str(_rows) + ', \n' + ' ' *20 + \
                   'cols = ' + str(_columns) + ', \n' + ' ' *20 +  \
                   'specs = [' + ', \n                            '.join([str(i) for i in _final_specs]) + '])\n\n\n' + \
                    '# ---------------------FIGURES --------------------- #\n\n' + \
                   '\n\n'.join(trace_code))
    
_add_graph.on_click(_copy_final_code)


In [8]:
display(_graphs_out)

Output(layout=Layout(align_items='center', border_bottom='12px inset teal', border_left='12px inset teal', bor…